In [ ]:
pip install xgboost pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import root_mean_squared_error
import numpy as np
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

path = 'Ready_to_train.csv'

# Load Concrete Compressive Strength dataset
data = pd.read_csv(path)


In [ ]:
# Drop samples with missing values
data.dropna(inplace=True)

# Get names of indexes for which column Age has value 30
indexNames = data[ data['difference_in_days'] == -1].index
# print(indexNames)
# Delete these row indexes from dataFrame
data.drop(indexNames , inplace=True)


data.loc[data.difference_in_days==0, 'difference_in_days'] = 1
data.loc[data.difference_in_days==2, 'difference_in_days'] = 1
data.loc[data.difference_in_days==3, 'difference_in_days'] = 1
data.loc[data.difference_in_days > 3, 'difference_in_days'] = 0

# Separate features and target variable
X = data.drop('difference_in_days', axis=1)
y = data['difference_in_days']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)



In [ ]:
# Initialize XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False)

# Define hyperparameters grid
param_grid = {
    'eta': [0.01, 0.1, 0.3],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'n_estimators': [100, 200, 300]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")


Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best Parameters: {'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.6}
Best Score: 0.7797937971892429


In [ ]:
# Predict and evaluate
y_pred = grid_search.best_estimator_.predict(X_test)


y_pred = [np.int64(element) for element in y_pred]



from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.86      0.77      0.81      9477
           1       0.68      0.79      0.73      5804

    accuracy                           0.78     15281
   macro avg       0.77      0.78      0.77     15281
weighted avg       0.79      0.78      0.78     15281

